In [1]:
from pikov import pikov
gamekitty = pikov.Pikov.open('gamekitty.pikov')

In [2]:
clip = gamekitty.get_clip(6)
clip

In [3]:
# Show the mirrored version of the same frames.
TO_MIRROR = 17
gamekitty.get_clip(6 + TO_MIRROR)

In [4]:
clip.frames[1]

In [5]:
clip.frames[0].image

In [6]:
gamekitty.get_image('md5-9b084a2f4cc7a7f940fac47ebe710c45')

In [7]:
sitting_clip = gamekitty.get_clip(1)
stand_up_clip = gamekitty.get_clip(2)
sit_and_paw_clip = gamekitty.get_clip(7)
sitting_clip + stand_up_clip + clip + clip + sit_and_paw_clip